In [1]:
import cv2

In [2]:
cameraID=0
webcam=cv2.VideoCapture(cameraID)

In [22]:
'''
Проверка сегментирования по изображению с камеры для отладки
'''
import numpy as np

stop=False
while stop==False:
    #frame=cv2.imread("Croped.png")
    ret,frame=webcam.read()
    
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    #ret, thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
    #thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    
    #img_erode = cv2.erode(thresh, np.ones((3, 3), np.uint8), iterations=1)
    #img_erode = cv2.erode(thresh, np.ones((3, 3), np.uint8), iterations=1)
    
    #img_erode = cv2.erode(gray, np.ones((2, 2), np.uint8), iterations=1)
    
    #blur = cv2.GaussianBlur(img_erode,(3,3),1)
    ret,thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    img_erode = cv2.erode(thresh, np.ones((10, 10), np.uint8), iterations=1)
    
    #cv2.imshow("Input", frame)
    #cv2.imshow("Blur", blur)
    #cv2.imshow("img_erode", img_erode)
    
    cv2.imshow("thresh", thresh)
    invertedImage = cv2.bitwise_not(img_erode)
    cv2.imshow("invertedImage", invertedImage)
    
    cv2.imshow("img_erode", img_erode)
    output = frame.copy()
    
    contours, hierarchy = cv2.findContours(invertedImage, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    for idx, contour in enumerate(contours):
        (x, y, w, h) = cv2.boundingRect(contour)
        # print("R", idx, x, y, w, h, cv2.contourArea(contour), hierarchy[0][idx])
        # hierarchy[i][0]: the index of the next contour of the same level
        # hierarchy[i][1]: the index of the previous contour of the same level
        # hierarchy[i][2]: the index of the first child
        # hierarchy[i][3]: the index of the parent
        #if hierarchy[0][idx][3] == 0:
            #color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
            
        cv2.rectangle(output, (x, y), (x + w, y + h), (100, 0, 0), 1)
            #cv.drawContours(drawing, contours, i, color, 2, cv.LINE_8, hierarchy, 0)
    cv2.imshow("output", output)
    key=cv2.waitKey(1)
    if(key==ord("q")):
                stop=True



cv2.destroyAllWindows()





In [3]:
'''
Сохранение в сегментированные картинки
'''

import numpy as np
def letters_extract(image_file, out_size=28):
    img = cv2.imread(image_file)
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    img_erode = cv2.erode(thresh, np.ones((10, 10), np.uint8), iterations=1)
    invertedImage = cv2.bitwise_not(img_erode)

    # Get contours
    contours, hierarchy = cv2.findContours(invertedImage, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    output = invertedImage.copy()

    letters = []
    for idx, contour in enumerate(contours):
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(output, (x, y), (x + w, y + h), (70, 0, 0), 1)
        letter_crop = invertedImage[y:y + h, x:x + w]
        # print(letter_crop.shape)

        # Resize letter canvas to square
        size_max = max(w, h)
        letter_square = 255 * np.zeros(shape=[size_max, size_max], dtype=np.uint8)
        if w > h:
            # Enlarge image top-bottom
            # ------
            # ======
            # ------
            y_pos = size_max//2 - h//2
            letter_square[y_pos:y_pos + h, 0:w] = letter_crop
        elif w < h:
            # Enlarge image left-right
            # --||--
            x_pos = size_max//2 - w//2
            letter_square[0:h, x_pos:x_pos + w] = letter_crop
        else:
            letter_square = letter_crop

        # Resize letter to 28x28 and add letter and its X-coordinate
        letters.append((x, w, cv2.resize(letter_square, (out_size, out_size), interpolation=cv2.INTER_AREA)))

    # Sort array in place by X-coordinate
    letters.sort(key=lambda x: x[0], reverse=False)

    return letters

from PIL import Image
letters=letters_extract("Croped.png",28)
for i in range(len(letters)):
    #print(letters[i])
    im = Image.fromarray(letters[i][2])
    im.save(str(i)+".jpeg")